In [265]:
!pip install sympy numpy matplotlib scipy


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [266]:
import numpy as np
import sympy as sp
import matplotlib as plt
import scipy as sc

This script aids the creation of a Sedra high pass notch filter
Setup:

In [267]:
# Desired parameters:
Q = 5.25
fp = 2791.63
fz = 742.24

# Op-Amp parameters
GBWP = 1000000

# since this is a high pass notch filter, n1 has to be 0 so that transmission zeros are possible
n1 = 0

# since the gain of the system when the frequency goes to infinity has to be 1 for it to be normalized, n2 has to be 1 over the overshoot Q
n2 = 0.416

1) The value of Qo has to be chosen according to the design parameters since it depends on the sensibility of the components.

In [268]:
Qo = 2

2) The parameters K, k and m are then calculated from the previous parameters

In [269]:
K = 1 + ((1/(2*(Qo**2)))*(1 - (Qo/Q)))
k = (n2 * ((fz/fp)**2)/(1 - (Qo/Q)))
m = k * ((K - 1)/K) * ( 1 + 2 * Qo**2 * (fp/fz)**2)

print("K: {}" .format(K))
print("k: {}".format(k))
print("m: {}".format(m))


K: 1.0773809523809523
k: 0.04750532240091258
m: 0.38953353144315916


Then, the corrected wp and Qp values are calculated

In [270]:
wp = 2*np.pi*fp*( 1 + Qo*(fp/GBWP))
print("Wp: {}".format(wp))

Qp = Q*(1 - 2*Qo*Q*(fp/GBWP)*((1/(2*Q)) - (fp/GBWP)))
print("Qp: {}".format(Qp))

Wp: 17638.26081413586
Qp: 5.221547085085773


Afterwards, the values of the capacitors are calculated. To do this, firstly the value of C21 is fixed to an arbitrary number. Then, the value of C22 is calculated by taking the previous nominal value of the result got from the m/m-1 ratio. Finally, C3 is equal to the sum of the two

In [271]:
# arbitrary C21 value
C21 = 20e-9
max_C22 = C21 * (m/(1 - m))

print("C21: {}".format(C21))
print("Max C22: {}".format(max_C22))

C21: 2e-08
Max C22: 1.2761832189211864e-08


Now the user has to pick a value for C22

In [272]:
C22 = 12e-9
max_C3 = C21 + C22
print("Max C3: {}".format(max_C3))

Max C3: 3.2e-08


Then, C3 has to be picked to the lower nominal value

In [273]:
C3 = 22e-9
print("C3: {}".format(C3))

C3: 2.2e-08


Then, the admittances R1 and R4 are calculated:

In [274]:
G1 =  ( 2 * Qo * wp * np.sqrt(C3*(C21 + C22)))
G4 =  G1 / (4*(Qo ** 2))

print("G1: {}".format(G1))
print("G4: {}".format(G4))

G1: 0.001871983778397351
G4: 0.00011699898614983443


Then, the user has to pick a value for Rb:

In [275]:
Rb = 100

Gb = 1/Rb

Ga = Gb * (( G4 / G1 ) * ((C21 + C22 + C3 )/C3) - (wp * (C21 + C22) / (Qp * G1)))

print("Ra: {}".format(1/Ga))

Ra: 1045.309889626347


Then the system of equations is solved for the values of Ga1, Ga2, G41 and G42

In [276]:
Ga1s, Ga2s, G41s, G42s, Gas, G4s, C3s, C22s, C21s, G1s = sp.symbols('Ga1, Ga2, G41, G42, Gas, G4, C3s, C22s, C21s, G1s')
Qps, wps, Gbs = sp.symbols('Qps, wps, Gbs')

# e1)

e1 = sp.Eq(Ga1s + Ga2s, Gas)
e1 = e1.subs(Gas, Ga)

# e2)

e2 = sp.Eq(G41s + G42s, G4s)
e2 = e2.subs(G4s, G4)

# e3)
lhs_1 = (G1s * G4s) / (C3s * (C21s + C22s)) * ((G42s/G4s)*((Gas + Gbs)/Gbs) - (Ga2s/Gbs))
lhs_2 = (((Gas + Gbs)/ Gbs)*(C22s/(C21s + C22s))) - (Ga2s/Gbs)
lhs = lhs_1/lhs_2
rhs = (2*np.pi*fz)**2

e3 = sp.Eq(rhs, lhs)

e3 = e3.subs(C3s, C3)
e3 = e3.subs(C21s, C21)
e3 = e3.subs(C22s, C22)
e3 = e3.subs(G4s, G4)
e3 = e3.subs(Gas, Ga)
e3 = e3.subs(Gbs, Gb)
e3 = e3.subs(G1s, G1)

# e3)
lhs = ((Gas + Gbs)/Gbs)*G42s*((1/(C21s + C22s)) + (1/C3s))
rhs = (Ga2s/Gbs)*((G1s/(C21s + C22s)) + G4s*((1/(C21s + C22s)) + 1/C3s))

e4 = sp.Eq(rhs, lhs)
e4 = e4.subs(C3s, C3)
e4 = e4.subs(C21s, C21)
e4 = e4.subs(C22s, C22)
e4 = e4.subs(G4s, G4)
e4 = e4.subs(Gas, Ga)
e4 = e4.subs(Gbs, Gb)
e4 = e4.subs(G1s, G1)

# print equations

#print("E1")
#sp.pprint(e1)
#print("E2")

#sp.pprint(e2)
#print("E3")

#sp.pprint(e3)
#print("E4")

#sp.pprint(e4)

solutions = sp.solve((e1, e2, e3, e4), (Ga1s, Ga2s, G41s, G42s))

Ra1 = 1/solutions[Ga1s]
Ra2 = 1/solutions[Ga2s]
R41 = 1/solutions[G41s]
R42 = 1/solutions[G42s]
Rb = 1/Gb

real_n2 = ((Ga + Gb)/Gb)*(C22/(C21 + C22)) - (solutions[Ga2s]/Gb)
print(solutions)

display("N2: {}".format(real_n2))
print("R1: {}".format(1/G1))
print("Ra1: {}".format(Ra1))
print("Rb: {}".format(Rb))
print("Ra2: {}".format(Ra2))
print("R41: {}".format(R41))
print("R42: {}".format(R42))
print("C21: {}".format(C21))
print("C22: {}".format(C22))
print("C3: {}".format(C3))


{G41: 0.000113498722535796, G42: 3.50026361403767e-6, Ga1: 0.000913056332243947, Ga2: 4.35977757138922e-5}


'N2: 0.406514751477030'

R1: 534.1926631736752
Ra1: 1095.22267650494
Rb: 100.0
Ra2: 22936.9499619072
R41: 8810.67185302117
R42: 285692.767821697
C21: 2e-08
C22: 1.2e-08
C3: 2.2e-08
